In [5]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
#from api_keys import p_key

# Incorporated citipy to determine city based on latitude and longitude
#from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/hotel.csv"

# Range of latitudes and longitudes
#lat_range = (-90, 90)
#lng_range = (-180, 180)

### API Calls

In [6]:
#Ryan API

In [7]:
#Nicole API

In [26]:
# Create aiport dataframe
city_df = pd.read_csv('.../City-Airport_Output.csv')
city_df.dropna(inplace = True) 
city_df.head()


FileNotFoundError: [Errno 2] No such file or directory: '.../City-Airport_Output.csv'

In [9]:
#Kayla API

import requests


url = "https://hotels-com-free.p.rapidapi.com/srle/listing/v1/brands/hotels.com"

querystring = {"lat":"37.788719679657554","lon":"-122.40057774847898","checkIn":"2021-03-15","checkOut":"2021-03-19","rooms":"2","locale":"en_US","currency":"USD","pageNumber":"1"}

headers = {
    'x-rapidapi-key': "7d7c570015msh06f88cde872d5e5p15c904jsn38364700fb06",
    'x-rapidapi-host': "hotels-com-free.p.rapidapi.com"
    }

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

response.json()

response = response.json()


{"result":"OK","data":{"body":{"header":"22 Anthony St, San Francisco, CA 94105, USA","query":{"destination":{"id":"1493604","value":"22 Anthony St, San Francisco, CA 94105, USA","resolvedLocation":"GEO_LOCATION:22 Anthony St, San Francisco, CA 94105, USA|37.78880310058594|-122.40035247802734:GEOCODE:UNKNOWN"}},"searchResults":{"totalCount":411,"results":[{"id":693927,"name":"Hotel Ikon","starRating":2.5,"urls":{},"address":{"streetAddress":"325 Sutter Street","extendedAddress":"","locality":"San Francisco","postalCode":"94108","region":"CA","countryName":"United States","countryCode":"us","obfuscate":false},"welcomeRewards":{"collect":true},"guestReviews":{"unformattedRating":3.3,"rating":"3.3","total":187,"scale":5,"badge":"good","badgeText":"Good"},"landmarks":[{"label":"22 Anthony St, San Francisco, CA 94105, USA","distance":"0.3 miles"},{"label":"San Francisco","distance":"0.2 miles"}],"geoBullets":[],"ratePlan":{"price":{"current":"$68","exactCurrent":68.31,"old":"$99","info":"ni

In [10]:
response


{'result': 'OK',
 'data': {'body': {'header': '22 Anthony St, San Francisco, CA 94105, USA',
   'query': {'destination': {'id': '1493604',
     'value': '22 Anthony St, San Francisco, CA 94105, USA',
     'resolvedLocation': 'GEO_LOCATION:22 Anthony St, San Francisco, CA 94105, USA|37.78880310058594|-122.40035247802734:GEOCODE:UNKNOWN'}},
   'searchResults': {'totalCount': 411,
    'results': [{'id': 693927,
      'name': 'Hotel Ikon',
      'starRating': 2.5,
      'urls': {},
      'address': {'streetAddress': '325 Sutter Street',
       'extendedAddress': '',
       'locality': 'San Francisco',
       'postalCode': '94108',
       'region': 'CA',
       'countryName': 'United States',
       'countryCode': 'us',
       'obfuscate': False},
      'welcomeRewards': {'collect': True},
      'guestReviews': {'unformattedRating': 3.3,
       'rating': '3.3',
       'total': 187,
       'scale': 5,
       'badge': 'good',
       'badgeText': 'Good'},
      'landmarks': [{'label': '22 Anth

In [22]:
counter = 0
cities = ["San Francisco"]
results=[]

    

#For loop
for city in cities:
    counter=counter+1  
    
    response = requests.request("GET", url, headers=headers, params=querystring)
    response = response.json()
    
    nh = len(response["data"]["body"]["searchResults"]["results"])
    
    for i in range(nh):
    
        Hotel = response["data"]["body"]["searchResults"]["results"][i]["name"]
        Rating = response["data"]["body"]["searchResults"]["results"][i]["starRating"]
        City = response["data"]["body"]["searchResults"]["results"][i]["address"]["locality"]
        State = response["data"]["body"]["searchResults"]["results"][i]["address"]["region"]
        Country =response["data"]["body"]["searchResults"]["results"][i]["address"]["countryCode"]
        Stay_Rate = response["data"]["body"]["searchResults"]["results"][i]["ratePlan"]["price"]["exactCurrent"]
        results.append([Hotel, Rating,City, State, Country,Stay_Rate])
print(results)
   
               
    


[['Hotel Ikon', 2.5, 'San Francisco', 'CA', 'us', 68.31], ['The Park Central San Francisco – Hyatt affiliated hotel', 4.0, 'San Francisco', 'CA', 'us', 119.7], ['Cornell Hotel de France', 3.5, 'San Francisco', 'CA', 'us', 118.47], ['Hotel Triton', 4.0, 'San Francisco', 'CA', 'us', 104.3], ['The Cartwright Hotel - Union Square BW Premier Collection', 3.5, 'San Francisco', 'CA', 'us', 88.5], ['Hayes Valley Inn', 3.0, 'San Francisco', 'CA', 'us', 102.13], ['The Clancy, Autograph Collection', 4.0, 'San Francisco', 'CA', 'us', 147.0], ['Mithila Hotel', 2.0, 'San Francisco', 'CA', 'us', 45.64]]


In [23]:
hotel_df = pd.DataFrame(results,columns=["Hotel","Rating","City", "State", "Country","Stay_Rate"])
hotel_df

,Hotel,Rating,City,State,Country,Stay_Rate
0,Hotel Ikon,2.5,San Francisco,CA,us,68.31
1,The Park Central San Francisco – Hyatt affilia...,4.0,San Francisco,CA,us,119.70
2,Cornell Hotel de France,3.5,San Francisco,CA,us,118.47
3,Hotel Triton,4.0,San Francisco,CA,us,104.30
4,The Cartwright Hotel - Union Square BW Premier...,3.5,San Francisco,CA,us,88.50
5,Hayes Valley Inn,3.0,San Francisco,CA,us,102.13
6,"The Clancy, Autograph Collection",4.0,San Francisco,CA,us,147.00
7,Mithila Hotel,2.0,San Francisco,CA,us,45.64
